In [22]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [14]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [15]:
df.drop('timestamp', axis=1, inplace=True)

rating_matrix = df.pivot(index='user_id', columns='item_id', values='rating')

print(rating_matrix.shape)  

(943, 1682)


In [24]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_matrix = train_df.pivot(index='user_id', columns='item_id', values='rating')

print(train_matrix.shape)

print(train_matrix.head())

(943, 1653)
item_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                              ...   
1         NaN   3.0   4.0   NaN   3.0   NaN   4.0   NaN   5.0   3.0  ...   
2         4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

item_id  1668  1670  1671  1672  1673  1676  1678  1679  1680  1681  
user_id                                                              
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5         NaN   NaN   NaN   NaN   N

In [25]:
filled_matrix = train_matrix.fillna(0)
user_similarity = cosine_similarity(filled_matrix)
user_similarity = pd.DataFrame(user_similarity, 
                               index=filled_matrix.index, 
                               columns=filled_matrix.index)

print(user_similarity.head())


user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.136196  0.030424  0.026203  0.284613  0.331412  0.319056   
2        0.136196  1.000000  0.114644  0.168220  0.093128  0.162165  0.095848   
3        0.030424  0.114644  1.000000  0.346894  0.000000  0.085071  0.032829   
4        0.026203  0.168220  0.346894  1.000000  0.011848  0.051287  0.075209   
5        0.284613  0.093128  0.000000  0.011848  1.000000  0.168527  0.298438   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.274139  0.083486  0.281396  ...  0.277459  0.084849  0.205849   
2        0.091360  0.149476  0.125701  ...  0.149359  0.268977  0.320095   
3        0.053875  0.060177  0.052552  ...  0.021713  0.017707  0.154299   
4        0.142100  0.060465  0.035202  ...  0.034908

In [26]:
def predict_rating(user_id, item_id, k=5):
    if item_id not in train_matrix.columns:
        return 3.0

    item_ratings = train_matrix[item_id].dropna()

    if user_id not in user_similarity:
        return item_ratings.mean()

    similarities = user_similarity.loc[user_id, item_ratings.index]
    top_k_users = similarities.nlargest(k)

    numerator = 0
    denominator = 0
    for sim_user, sim_score in top_k_users.items():
        rating = train_matrix.at[sim_user, item_id]
        if np.isnan(rating):
            continue
        numerator += sim_score * rating
        denominator += abs(sim_score)

    if denominator == 0:
        return item_ratings.mean()

    return numerator / denominator


In [ ]:
predictions = []
truths = []

for _, row in test_df.iterrows():
    user, item, true_rating = row['user_id'], row['item_id'], row['rating']
    pred_rating = predict_rating(user, item, k=5)
    predictions.append(pred_rating)
    truths.append(true_rating)

rmse = math.sqrt(mean_squared_error(truths, predictions))
print("RMSE:", rmse)


RMSE: 1.0613934416462578
